# Import packages

In [1]:
%load_ext autoreload
%autoreload 2

from vrgaze.parser import get_args_parser, set_paths
from vrgaze.vrgaze import vrGazeCore, vrGazeData

import numpy as np

import glob
from matplotlib import pyplot as plt
import cv2
import numpy as np
import os, sys

import time

# Set arguments to vrGazeCore

Set base directory, project directory, raw data directory, stimuli directory, & processing steps

In [2]:
base_dir = '/Users/deepasriprasad/Documents/pyGazeCore/'

args = [
    f'--project_dir={base_dir}',
    f'--raw_data_folder=rawData',
    f'--stim_folder=stimuli/combined/',
    f'--plot_fixations',
    f'--plot_density_maps',
    f'--heatmap_timesteps=1',
]

Load arguments into vrGazeCore

In [3]:
parser = get_args_parser()

params = parser.parse_args(args)
paths = set_paths(params)

Check that the following paths are correct:
project_raw_data_dir = /Users/deepasriprasad/Documents/pyGazeCore/rawData
project_stim_dir = /Users/deepasriprasad/Documents/pyGazeCore/stimuli/combined/


Set and load subject data

In [ ]:
# parsed_data[94].get_density_map

In [4]:
subject_fn = 'furrow381_V1.txt'
subject = os.path.splitext(subject_fn)[0]

In [5]:
vrGaze = vrGazeCore(params, paths)

raw_data = vrGaze.loadRawData(subject_fn)
raw_data = vrGaze.processRawData(raw_data)
parsed_data = vrGaze.parseTrials(data=raw_data, subject=subject)

Averaging coordinates from both eyes!


# Test vrgc on Single Subject

In [ ]:
my_trial = parsed_data[0]
print(my_trial.trial_name)

# my_trial = vrGaze.runFindFixations(my_trial)

In [ ]:
import time
print(f'Running {len(parsed_data)} trials')

start_time = time.time()

for i, trial in enumerate(parsed_data):
    print(f"Running {trial.trial_name} (Trial Number {i})")  
    
    trial = vrGaze.runFindFixations(trial)
    
    trial = vrGaze.runHeatmapping(trial)
    
    print (time.time() - start_time)

In [ ]:
my_trial = parsed_data[12]
image_path = my_trial.get_image_path()

vrGaze.plotFixations(my_trial.get_fixations(), image_path, fig_size=(12,6))

In [ ]:
image_path = my_trial.get_image_path()

vrGaze.params.heatmap_timesteps = 1

split_timecourse = vrGaze.splitFixationTimesteps(my_trial.get_fixations())
density_map = vrGaze.createDensityMap(split_timecourse[-1])
vrGaze.plotFixationDensity(density_map, image_path, fig_size=(12,6))

# Test loading group data

In [6]:
subjects = ['furrow012_V1','furrow380_V1', 'furrow381_V1']

group_trials = vrGaze.loadGroupFixations(subjects)

print (f'Found {len(group_trials)} files shared across {len(subjects)} subjects')

vrGaze.params.cohort_name = "testCohort"

Found 61 files shared across 3 subjects


In [7]:
print(group_trials)

In [ ]:
for trials in group_trials:
    [vrGaze.runHeatmapping(trial) for trial in trials]
    group_data = vrGaze.runHeatmapping(trials)

# Test of LR wrap around 

In [ ]:
import pandas as pd

sample = np.array([
    [15, 170, 0.5],
    [345, 10, 0.5],
    [180, 90, 0.5],
])

df_test = pd.DataFrame(sample, columns=['fix_yaw', 'fix_pitch', 'duration'])

density_map = vrGaze.createDensityMap(df_test)
plt.imshow(density_map)

# Test of variable width filter

In [ ]:
sample = np.array([
    [180, 170, 0.5],
    [180, 130, 0.5],
    [180, 90, 0.5],
    [180, 50, 0.5],
    [180, 10, 0.5],
])

df_test = pd.DataFrame(sample, columns=['fix_yaw', 'fix_pitch', 'duration'])

density_map = vrGaze.createDensityMap(df_test)
plt.imshow(density_map)

# Correlate data between python and matlab

In [ ]:
subjects = ['furrow012_V1']

subject_trials = vrGaze.loadGroupFixations(subjects)

# print (f'Found {len(data)} files shared across {len(subjects)} subjects')

In [ ]:
from scipy.io import loadmat

matlab_base_dir = '~Documents/matGazeCore/eyeTrackResults/heatMaps/mat/'

corrs = []

for i, trial in enumerate(subject_trials):
    print (f'Trial {i+1}/{len(subject_trials)}')
    current_trial = trial[0]
    matlab_dir = os.path.join(matlab_base_dir, current_trial.subject)
    
    matlab_path = glob.glob(os.path.join(matlab_dir, f'*{current_trial.trial_name}*'))[0]
    
    mat = loadmat(matlab_path)
    matlab_density_map = mat['timestepHeatmaps'][0][0]
    
    density_map = vrGaze.createDensityMap(current_trial.get_fixations())
    corr = np.corrcoef(matlab_density_map.flatten(), density_map.flatten())[0][1]
    corrs.append(corr)

In [ ]:
print(corrs)

In [ ]:
import seaborn as sns
sns.boxplot(corrs, palette='BuPu')
sns.despine()
plt.title(f'Correlation between matlab and python (N={sum(~np.isnan(corrs))} images)')

In [ ]:

# trial = parsed_data[trial_number].get_raw_data()

# first run the confidence filter
confidence_filter, percent_removed = vrGaze.confidenceFilter(trial)

# trial.set_confidence_filter(confidence_filter)

# ADD CONFIDENCE FLAG HERE

# next eccentricity filter
# RUN ECCENTRICITY FLAG HERE

eccentricity_filter, percent_removed = vrGaze.eccentricityFilter(trial)

# trial.set_eccentricity_filter(eccentricity_filter)

# 

# Maps eye coordinates on screen to FOV, roll to radians
preproc = vrGaze.preprocessTrialData(trial)

# Connects head and eye position to create gaze on sphere
# ONLY NECESSARY IF EYETRACKING
# rectified = vrGaze.rectifyGaze(preproc)

sys.exit(0)

trial_filtered = trial.loc[~np.logical_or(confidence_filter, eccentricity_filter)]
trial_filtered = trial_filtered.loc[1:]


trial_fixations = vrGaze.calculateFixations(trial_filtered)


trial_name = sorted(trial['trial'].unique())[0]
image_path = glob.glob(os.path.join(vrGaze.paths['project_stim_dir'], f'{trial_name}*'))[0]

# plt.colorbar(orientation="horizontal")
# TODO: Wrap points Equirect 
## INSERT WRAP POINTS

# TODO: Bilateral filtering
# INSERT BILATERAL FILTERING

# TODO: Interpolation
## INSERT INTERPOLATION

In [ ]:
vrGaze.plotFixations(trial_fixations, image_path)

In [ ]:
density_map = vrGaze.createDensityMap(trial_fixations)
vrGaze.plotFixationDensity(density_map, image_path)

In [ ]:
# load in pkl file
import pickle

f = open('/Users/deepasriprasad/Documents/pyGazeCore/eyeTrackResults/heatMaps/pkl/furrow012_V1/furrow012_V1_3P_4230958772_bdfb3f3af4_o_00051.pkl', 'rb')   # 'rb' for reading binary file
mydict = pickle.load(f)     
f.close()                       

print(mydict)

# Test Sanity Targets

In [2]:
# set base params & paths

base_dir = '/Users/deepasriprasad/Documents/pyGazeCore/'
exclude_scenes = ['practice1','practice2']
pretrial_list = ['fixate', 'sanityTarget']

args = [
    f'--project_dir={base_dir}',
    f'--raw_data_folder=rawData',
    f'--stim_folder=stimuli/combined/',
    f'--plot_fixations',
    f'--plot_density_maps',
    f'--heatmap_timesteps=1',
    f'--make_density_map_gif'
]

parser = get_args_parser()

params = parser.parse_args(args)

for item in pretrial_list:
    params.pretrial_list.append(item)
    
for scene in exclude_scenes:
    params.exclude_scenes.append(scene)
    
paths = set_paths(params)

vrGaze = vrGazeCore(params, paths)

Check that the following paths are correct:
project_raw_data_dir = /Users/deepasriprasad/Documents/pyGazeCore/rawData
project_stim_dir = /Users/deepasriprasad/Documents/pyGazeCore/stimuli/combined/


In [3]:
# list heatmap timesteps

# list subject filenames
subject_fn = 'furrow012_V1.txt'


subject = os.path.splitext(subject_fn)[0]

raw_data = vrGaze.loadRawData(subject_fn)
raw_data = vrGaze.processRawData(raw_data)
parsed_data = vrGaze.parseTrials(data=raw_data, subject=subject)
parsed_data_key = vrGaze.parsedDataKey(parsed_data)

Averaging coordinates from both eyes!


In [4]:
trial = parsed_data[15]
print(trial.trial_name)
image_path = '/Users/deepasriprasad/Documents/pyGazeCore/stimuli/combined/_sanityTarget360.png'
trial.image_path = image_path
print(trial.image_path)

_sanityTarget360_0000
/Users/deepasriprasad/Documents/pyGazeCore/stimuli/combined/_sanityTarget360.png


In [5]:
trial = vrGaze.runFindFixations(trial)
trial.get_fixations()

CONFIDENCE FILTER - percent removed: 9.46%
ECCENTRICITY FILTER - percent removed: 0.00%
FIXATION CONCATENATION: concatenated 0 out of 10 fixations
FIXATION TRIM FILTER - removed 1 out of 10 fixations
FIXATION DURATION FILTER - removed 2 out of 9 fixations

FIXATION STATISTICS
Num fixations: 7
Avg fixation duration: 1.211 seconds
STD fixation duration: 1.671 seconds
Min fixation duration: 0.132 seconds
Max fixation duration: 4.922 seconds


,fix_yaw,fix_pitch,start_time,end_time,spread,duration,norm_start_time,norm_end_time
0,180.074939,89.665916,630.3656,631.0444,0.761719,0.6788,0.3996,1.0784
1,167.336562,85.594649,631.9080,633.0159,1.473411,1.1079,1.9420,3.0499
2,185.637951,88.356592,633.1505,633.2828,4.221000,0.1323,3.1845,3.3168
3,167.546308,84.522229,633.4297,634.3375,0.789800,0.9078,3.4637,4.3715
4,164.245088,87.456494,634.8578,635.2169,3.816681,0.3591,4.8918,5.2509
5,167.174503,85.236445,635.5532,635.9242,1.866194,0.3710,5.5872,5.9582
6,180.359716,88.323314,636.1104,641.0323,0.224014,4.9219,6.1444,11.0663


# Testing Time Segmenting & GIF Making

In [ ]:
# make time segmented 
base_dir = '/Users/deepasriprasad/Documents/pyGazeCore/'
exclude_scenes = ['practice1','practice2']
pretrial_list = ['fixate', 'sanityTarget']

args = [
    f'--project_dir={base_dir}',
    f'--raw_data_folder=rawData',
    f'--stim_folder=stimuli/combined/',
    f'--plot_fixations',
    f'--plot_density_maps',
    f'--heatmap_timesteps=1',
    f'--make_density_map_gif'
]

parser = get_args_parser()

params = parser.parse_args(args)

for item in pretrial_list:
    params.pretrial_list.append(item)
    
for scene in exclude_scenes:
    params.exclude_scenes.append(scene)
    
paths = set_paths(params)

subject_fn = 'furrow012_V1.txt'
subject = os.path.splitext(subject_fn)[0]



In [ ]:
vrGaze = vrGazeCore(params, paths)

raw_data = vrGaze.loadRawData(subject_fn)
raw_data = vrGaze.processRawData(raw_data)
parsed_data = vrGaze.parseTrials(data=raw_data, subject=subject)



In [ ]:
import time
print(f'Running {len(parsed_data)} trials')

start_time = time.time()

for i, trial in enumerate(parsed_data):
    # if manually excluded, skip trial
    if any([scene in trial.trial_name for scene in vrGaze.params.exclude_scenes]):
            print (f"Manually excluded {trial.trial_name} (Trial Number {i}). Skipping!")
            continue
    
    print(f"Running {trial.trial_name} (Trial Index {i})")  
     
    trial = vrGaze.runFindFixations(trial)
    
    trial = vrGaze.runHeatmapping(trial)
    
    print (time.time() - start_time)


In [ ]:
print(parsed_data[4].trial_number)
print(parsed_data[4].trial_name)

test_trial = parsed_data[4]
test_trial.get_fixations()
vrGaze.params.heatmap_timesteps = 32
test_trial = vrGaze.runHeatmapping(test_trial)

In [ ]:
test_trial = vrGaze.runFindFixations(test_trial)
test_trial = vrGaze.runHeatmapping(test_trial)

In [ ]:
df_fixations = test_trial.get_fixations()
df_timesteps = vrGaze.splitFixationTimesteps(df_fixations)
if df_timesteps is None or not any(list(map(np.size, df_timesteps))):
    print('whoops, something did not work!')

In [ ]:
print(len(df_timesteps))
print(df_timesteps[5])

In [ ]:
if np.any(df_timesteps[2]) == False:
    print('empty!')

In [ ]:
test_trial = vrGaze.runHeatmapping(test_trial)     

In [ ]:
# function that data frame of trial names, trial number, and idx in parsed_data
import pandas as pd

def parsedDataKey(parsed_data):
    parsed_data_key = pd.DataFrame({
        'parsed_data_idx': [i for i in range(len(parsed_data))],
        'trial_name': [trial.trial_name for trial in parsed_data],
        'trial_number': [trial.trial_number for trial in parsed_data]
    })
    
    return parsed_data_key

In [ ]:
parsed_data_key = parsedDataKey(parsed_data)
print(parsed_data_key)

# Full Runthrough let's go!!
1. Loop through subjects
2. Multiple time-steps
3. group heatmapping & gif making

In [ ]:
# set base params & paths

base_dir = '/Users/deepasriprasad/Documents/pyGazeCore/'
exclude_scenes = ['practice1','practice2']
pretrial_list = ['fixate', 'sanityTarget']

args = [
    f'--project_dir={base_dir}',
    f'--raw_data_folder=rawData',
    f'--stim_folder=stimuli/combined/',
    f'--plot_fixations',
    f'--plot_density_maps',
    f'--heatmap_timesteps=1',
    f'--make_density_map_gif'
]

parser = get_args_parser()

params = parser.parse_args(args)

for item in pretrial_list:
    params.pretrial_list.append(item)
    
for scene in exclude_scenes:
    params.exclude_scenes.append(scene)
    
paths = set_paths(params)

vrGaze = vrGazeCore(params, paths)

In [ ]:
# list heatmap timesteps

# list subject filenames
subject_fn_list = ['furrow380_V1.txt', 'furrow381_V1.txt']

# loop through individual subjects
for subject_fn in subject_fn_list:
    
    subject = os.path.splitext(subject_fn)[0]
    
    print(f'Running {subject}')
    
    raw_data = vrGaze.loadRawData(subject_fn)
    raw_data = vrGaze.processRawData(raw_data)
    parsed_data = vrGaze.parseTrials(data=raw_data, subject=subject)
    parsed_data_key = vrGaze.parsedDataKey(parsed_data)
    
    print(f'Running {len(parsed_data)} trials')
    
    start_time = time.time()
    
    for i, trial in enumerate(parsed_data):
        # if manually excluded, skip trial
        if any([scene in trial.trial_name for scene in vrGaze.params.exclude_scenes]):
            print (f"Manually excluded {trial.trial_name} (Trial Number {i}). Skipping!")
            continue
        
        print(f"Running {trial.trial_name} (Trial Index {i})")  
        
        trial = vrGaze.runFindFixations(trial)
        trial = vrGaze.runHeatmapping(trial)
        
        print (time.time() - start_time)

In [ ]:
# group analysis
subject_fn_list = ['furrow012_V1.txt','furrow380_V1.txt', 'furrow381_V1.txt']

subjects = [os.path.splitext(subject)[0] for subject in subject_fn_list]
print(subjects)
group_trials = vrGaze.loadGroupFixations(subjects)

print (f'Found {len(group_trials)} files shared across {len(subjects)} subjects')

vrGaze.params.cohort_name = "testCohort"
vrGaze.params.heatmap_timesteps = 4




In [ ]:
for trials in group_trials:
    [vrGaze.runHeatmapping(trial) for trial in trials]
    group_data = vrGaze.runHeatmapping(trials)


# Meta

## TODO:
- loading in data as parsed_data ?
- higher time segments missing fixations??????
- get files from directory name
- matlab gif making
- delete unused code (commented out)
- convert to csv/make sure can read all data output equivalent to mat (don't know if need to do this)
- save something to logs

## Questions for group
directory file labeling (time segment folders) and structure
sanityTarget360 - uniform name in field?